In [158]:
import json
import pandas as pd
import numpy as np
import re # import dependency for Regular Expressions (regex)

In [159]:
file_dir = 'C://Users/manoh/iCloudDrive/University of Toronto/Data Analytics Bootcamp/Python & PostgreSQL - Module 8 - ETL/'

In [160]:
# Instead of reading the JSON wikipedia-movies.json into a DataFrame, we will perform data cleaning with it as a list of dictionaries, its natural form

# Create a context manager and open the JSON within this context:
# Use the load method from the json library to load the json into a list of dictionaries
with open(f'{file_dir}/wikipedia-movies.json', mode='r') as file:
    wiki_movies_raw = json.load(file)

7311

In [161]:
# It is best practice to examine our data by lookign at the first, last, and middle sets of data
# Since our JSON is pushed to a list of dictionaries, we can use list slicing to pull subsets of data
# First 5 records
wiki_movies_raw[:5]

[{'url': 'https://en.wikipedia.org/wiki/The_Adventures_of_Ford_Fairlane',
  'year': 1990,
  'imdb_link': 'https://www.imdb.com/title/tt0098987/',
  'title': 'The Adventures of Ford Fairlane',
  'Directed by': 'Renny Harlin',
  'Produced by': ['Steve Perry', 'Joel Silver'],
  'Screenplay by': ['David Arnott', 'James Cappe', 'Daniel Waters'],
  'Story by': ['David Arnott', 'James Cappe'],
  'Based on': ['Characters', 'by Rex Weiner'],
  'Starring': ['Andrew Dice Clay',
   'Wayne Newton',
   'Priscilla Presley',
   'Lauren Holly',
   'Morris Day',
   'Robert Englund',
   "Ed O'Neill"],
  'Narrated by': 'Andrew "Dice" Clay',
  'Music by': ['Cliff Eidelman', 'Yello'],
  'Cinematography': 'Oliver Wood',
  'Edited by': 'Michael Tronick',
  'Productioncompany ': 'Silver Pictures',
  'Distributed by': '20th Century Fox',
  'Release date': ['July 11, 1990', '(', '1990-07-11', ')'],
  'Running time': '102 minutes',
  'Country': 'United States',
  'Language': 'English',
  'Budget': '$20 million',


In [162]:
# Last 5 records
wiki_movies_raw[-5:]

[{'url': 'https://en.wikipedia.org/wiki/Holmes_%26_Watson',
  'year': 2018,
  'imdb_link': 'https://www.imdb.com/title/tt1255919/',
  'title': 'Holmes & Watson',
  'Directed by': 'Etan Cohen',
  'Produced by': ['Will Ferrell',
   'Adam McKay',
   'Jimmy Miller',
   'Clayton Townsend'],
  'Screenplay by': 'Etan Cohen',
  'Based on': ['Sherlock Holmes',
   'and',
   'Dr. Watson',
   'by',
   'Sir Arthur Conan Doyle'],
  'Starring': ['Will Ferrell',
   'John C. Reilly',
   'Rebecca Hall',
   'Rob Brydon',
   'Steve Coogan',
   'Ralph Fiennes'],
  'Music by': 'Mark Mothersbaugh',
  'Cinematography': 'Oliver Wood',
  'Edited by': 'Dean Zimmerman',
  'Productioncompanies ': ['Columbia Pictures',
   'Gary Sanchez Productions',
   'Mosaic Media Group',
   'Mimran Schur Pictures'],
  'Distributed by': 'Sony Pictures Releasing',
  'Release date': ['December 25, 2018',
   '(',
   '2018-12-25',
   ')',
   '(United States)'],
  'Running time': '90 minutes',
  'Country': 'United States',
  'Language

In [163]:
# Some records in the middle
wiki_movies_raw[3600:3605]

[{'url': 'https://en.wikipedia.org/wiki/Benji:_Off_the_Leash!',
  'year': 2004,
  'imdb_link': 'https://www.imdb.com/title/tt0315273/',
  'title': 'Benji: Off the Leash!',
  'Directed by': 'Joe Camp',
  'Written by': 'Joe Camp',
  'Starring': ['Benji', 'Nick Whitaker', 'Shaggy', 'Gypsy the Cockatoo'],
  'Music by': 'Antonio di Lorenzo',
  'Productioncompany ': 'Mulberry Square Productions',
  'Distributed by': 'Mulberry Square Productions',
  'Release date': ['March 26, 2004', '(', '2004-03-26', ')'],
  'Running time': '97 min',
  'Country': 'United States',
  'Language': 'English',
  'Box office': '$3,817,362'},
 {'url': 'https://en.wikipedia.org/wiki/The_Best_Thief_in_the_World',
  'year': 2004,
  'imdb_link': 'https://www.imdb.com/title/tt0389796/',
  'title': 'The Best Thief in the World',
  'Directed by': 'Jacob Kornbluth',
  'Produced by': ['Tim Perrell', 'Nicola Usborne'],
  'Written by': 'Jacob Kornbluth',
  'Starring': ['Marc Rozendaal',
   'Michael Silverman',
   'David Warsh

In [164]:
# Convert wiki data into dataframe
wiki_movies_df = pd.DataFrame(wiki_movies_raw)

In [165]:
wiki_movies_df.head()

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Predecessor,Founders,Area served,Products,Services,Russian,Hebrew,Revenue,Operating income,Polish
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990.0,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,Renny Harlin,"[Steve Perry, Joel Silver]","[David Arnott, James Cappe, Daniel Waters]","[David Arnott, James Cappe]","[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990.0,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet",James Foley,"[Ric Kidney, Robert Redlin]","[James Foley, Robert Redlin]",NaN,"[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://en.wikipedia.org/wiki/Air_America_(film),1990.0,https://www.imdb.com/title/tt0099005/,Air America,Roger Spottiswoode,Daniel Melnick,"[John Eskow, Richard Rush]",NaN,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990.0,https://www.imdb.com/title/tt0099012/,Alice,Woody Allen,Robert Greenhut,NaN,NaN,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990.0,https://www.imdb.com/title/tt0099018/,Almost an Angel,John Cornell,John Cornell,NaN,NaN,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [166]:
wiki_movies_df.columns.tolist()

['url',
 'year',
 'imdb_link',
 'title',
 'Directed by',
 'Produced by',
 'Screenplay by',
 'Story by',
 'Based on',
 'Starring',
 'Narrated by',
 'Music by',
 'Cinematography',
 'Edited by',
 'Productioncompany ',
 'Distributed by',
 'Release date',
 'Running time',
 'Country',
 'Language',
 'Budget',
 'Box office',
 'Written by',
 'Genre',
 'Theme music composer',
 'Country of origin',
 'Original language(s)',
 'Producer(s)',
 'Editor(s)',
 'Production company(s)',
 'Original network',
 'Original release',
 'Productioncompanies ',
 'Executive producer(s)',
 'Production location(s)',
 'Distributor',
 'Picture format',
 'Audio format',
 'Voices of',
 'Followed by',
 'Composer(s)',
 'Created by',
 'Also known as',
 'Opening theme',
 'No. of episodes',
 'Preceded by',
 'Author',
 'Publisher',
 'Publication date',
 'Media type',
 'Pages',
 'ISBN',
 'OCLC',
 'LC Class',
 'Cover artist',
 'Series',
 'Set in',
 'Adaptation by',
 'Suggested by',
 'Biographical data',
 'Born',
 'Died',
 'Resti

In [167]:
# 2. Plan
# Since we have 193 columns, as described above, we should modify our original JSON BEFORE casting to DataFrame to filter out unnecessary columns

# Create a list comprehension to only return movies with both a Director, and an IMDB link
# Recall syntax for list comprehension with a conditional is: [expression for element in <source_list> if <filter_expression>]
wiki_movies = [movie for movie in 
                wiki_movies_raw if ('Director' in movie or 'Directed by' in movie) 
                and 'imdb_link' in movie
                ]

# Check the new length of movies in our dataset
len(wiki_movies)

7080

In [168]:
# 1. (Re)-Inspect
# Cast our new filtered JSON output to a dataframe
pd.DataFrame(wiki_movies).head()

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Hepburn,Literally,Cantonese,Chinese,Yiddish,Arabic,Romanized,Russian,Hebrew,Polish
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,Renny Harlin,"[Steve Perry, Joel Silver]","[David Arnott, James Cappe, Daniel Waters]","[David Arnott, James Cappe]","[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet",James Foley,"[Ric Kidney, Robert Redlin]","[James Foley, Robert Redlin]",NaN,"[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://en.wikipedia.org/wiki/Air_America_(film),1990,https://www.imdb.com/title/tt0099005/,Air America,Roger Spottiswoode,Daniel Melnick,"[John Eskow, Richard Rush]",NaN,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990,https://www.imdb.com/title/tt0099012/,Alice,Woody Allen,Robert Greenhut,NaN,NaN,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990,https://www.imdb.com/title/tt0099018/,Almost an Angel,John Cornell,John Cornell,NaN,NaN,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [169]:
# No of episodes indicates that these rows might signify TV shows. We will modify the original list comprehension to eliminate these

wiki_movies = [movie for movie in wiki_movies_raw
               if ('Director' in movie or 'Directed by' in movie)
                   and 'imdb_link' in movie
                   and 'No. of episodes' not in movie]
pd.DataFrame(wiki_movies).head()

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Hepburn,Literally,Cantonese,Chinese,Yiddish,Arabic,Romanized,Russian,Hebrew,Polish
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,Renny Harlin,"[Steve Perry, Joel Silver]","[David Arnott, James Cappe, Daniel Waters]","[David Arnott, James Cappe]","[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet",James Foley,"[Ric Kidney, Robert Redlin]","[James Foley, Robert Redlin]",NaN,"[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://en.wikipedia.org/wiki/Air_America_(film),1990,https://www.imdb.com/title/tt0099005/,Air America,Roger Spottiswoode,Daniel Melnick,"[John Eskow, Richard Rush]",NaN,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990,https://www.imdb.com/title/tt0099012/,Alice,Woody Allen,Robert Greenhut,NaN,NaN,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990,https://www.imdb.com/title/tt0099018/,Almost an Angel,John Cornell,John Cornell,NaN,NaN,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [170]:
def clean_movie(movie):
    # Since each movie is a dictionary (JSONs are lists of dictionaries), we want to perform a non-destructive edit during our function
    # Therefore, take a copy of the movie
    movie = dict(movie) # create a non-destructive copy of movies. Reassigning it in local scope does not affect global scope
    # return our newly copied movie in local scope
    return movie

    


In [171]:
wiki_movies_df[wiki_movies_df['Arabic'].notnull()]


,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Predecessor,Founders,Area served,Products,Services,Russian,Hebrew,Revenue,Operating income,Polish
7060,https://en.wikipedia.org/wiki/The_Insult_(film),2018.0,https://www.imdb.com/title/tt7048622/,The Insult,Ziad Doueiri,"[Rachid Bouchareb, Jean Bréhat, Julie Gayet, A...",NaN,NaN,NaN,"[Adel Karam, Kamel El Basha]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7293,https://en.wikipedia.org/wiki/Capernaum_(film),2018.0,https://www.imdb.com/title/tt8267604/,Capernaum,Nadine Labaki,"[Michel Merkt, Khaled Mouzanar]","[Nadine Labaki, Jihad Hojaily, Michelle Keserw...","[Georges Khabbaz, Nadine Labaki, Michelle Kese...",NaN,"[Zain Al Rafeea, Yordanos Shiferaw, Boluwatife...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [172]:
# Inspect all the columns in our dataframe
# Chaining the tolist() method to the columns attribute and sorting the list returns a list of column names in alphabetical order
sorted(wiki_movies_df.columns.tolist())

['Actor control',
 'Adaptation by',
 'Alias',
 'Alma mater',
 'Also known as',
 'Animation by',
 'Arabic',
 'Area',
 'Area served',
 'Artist(s)',
 'Attraction type',
 'Audio format',
 'Author',
 'Based on',
 'Biographical data',
 'Bopomofo',
 'Born',
 'Box office',
 'Budget',
 'Camera setup',
 'Cantonese',
 'Characters',
 'Children',
 'Chinese',
 'Cinematography',
 'Closing date',
 'Color process',
 'Comics',
 'Composer(s)',
 'Coordinates',
 'Country',
 'Country of origin',
 'Cover artist',
 'Created by',
 'Date premiered',
 'Designer(s)',
 'Developed by',
 'Developer(s)',
 'Dewey Decimal',
 'Died',
 'Directed by',
 'Director',
 'Distributed by',
 'Distributor',
 'Divisions',
 'Duration',
 'Edited by',
 'Editor(s)',
 'Ending theme',
 'Engine',
 'Engine(s)',
 'Executive producer(s)',
 'Family',
 'Fate',
 'Film(s)',
 'Followed by',
 'Format(s)',
 'Formerly',
 'Founded',
 'Founder',
 'Founders',
 'French',
 'Full name',
 'Gender',
 'Genre',
 'Genre(s)',
 'Genres',
 'Gwoyeu Romatzyh',
 'Ha

In [173]:
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    alt_titles = {} # initialize an empty dictionary to hold our alternate titles
    # pass in a list of alternate title keys, where foreign characters are often appended (or additional data)
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        # Check if the key in the list above is a key in the movie we are cleaning, then run the following
        if key in movie:
            # Add the key and its value to the alt_titles dictionary
            alt_titles[key] = movie[key]
            # Remove the key and value from the movie
            movie.pop(key)
    # Once our for loop is done, check if there are any key:value pairs in our alt_titles dictionary
    if len(alt_titles) > 0:
        # if there are key:value pairs in alt_titles, add the dictionary and its value as a key to the (local) movie dictionary
        movie['alt_titles'] = alt_titles
    
    # Transferring values in an old column to a new column name
    def change_column_name(old_name, new_name):
        if old_name in movie:
            # Since pop() returns the value, we can use this to pass it as the value for a new key in the movie dictionary with name new_name
            movie[new_name] = movie.pop(old_name)
    # Using the above cleaning function, convert the Directed by column to Director        
    change_column_name('Directed by', 'Director')

    # Change additional column names to consolidate
    change_column_name('Adaptation by', 'Writer(s)')
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Original release', 'Release date')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Released', 'Release Date')
    change_column_name('Release Date', 'Release date')
    change_column_name('Screen story by', 'Writer(s)')
    change_column_name('Screenplay by', 'Writer(s)')
    change_column_name('Story by', 'Writer(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Written by', 'Writer(s)')


    # Once cleaning is complete, return the value of the local movie dictionary
    return movie


In [174]:
# Using a list comprehension, create a list of cleaned movies by passing in the above created function

clean_movies = [clean_movie(movie) for movie in wiki_movies_raw]

# Create a dataframe from the clean_movies input
wiki_movies_df = pd.DataFrame(clean_movies)

In [175]:
# Check our newly created column name list
sorted(wiki_movies_df.columns.tolist())

['Actor control',
 'Alias',
 'Alma mater',
 'Animation by',
 'Area',
 'Area served',
 'Artist(s)',
 'Attraction type',
 'Audio format',
 'Author',
 'Based on',
 'Biographical data',
 'Bopomofo',
 'Born',
 'Box office',
 'Budget',
 'Camera setup',
 'Characters',
 'Children',
 'Cinematography',
 'Closing date',
 'Color process',
 'Comics',
 'Composer(s)',
 'Coordinates',
 'Country',
 'Cover artist',
 'Created by',
 'Date premiered',
 'Designer(s)',
 'Developed by',
 'Developer(s)',
 'Dewey Decimal',
 'Died',
 'Director',
 'Distributor',
 'Divisions',
 'Duration',
 'Editor(s)',
 'Ending theme',
 'Engine',
 'Engine(s)',
 'Executive producer(s)',
 'Family',
 'Fate',
 'Film(s)',
 'Followed by',
 'Format(s)',
 'Formerly',
 'Founded',
 'Founder',
 'Founders',
 'Full name',
 'Gender',
 'Genre',
 'Genre(s)',
 'Genres',
 'Gwoyeu Romatzyh',
 'Hanyu Pinyin',
 'Headquarters',
 'Height',
 'Hokkien POJ',
 'IPA',
 'ISBN',
 'Illustrator',
 'Industry',
 'Instruments',
 'Jyutping',
 'Key people',
 'LC Cla

In [176]:
# Removing duplicate rows using Regular Expressions (regex)

# use the str.extract() method to pull certain values that match the regular expression

'''	- R before the string indicates that that the regex is a raw string of text
	- Tt indicates matching two lowercase t's
	- \d indicates matching a numerical digit
{7} says to match the last item (\d) 7 times'''

wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
print(len(wiki_movies_df))

# Use the drop duplicates method to drop duplicate rows of the dataframe, subset on column id 'imdb_id'
# inplace means that drop_duplicates will drop them from the existing df, instead of returning a copy
wiki_movies_df.drop_duplicates(subset='imdb_id', inplace=True)
print(len(wiki_movies_df))
wiki_movies_df.head()


7311
7056


,url,year,imdb_link,title,Based on,Starring,Narrated by,Cinematography,Release date,Running time,...,Divisions,Subsidiaries,Predecessor,Founders,Area served,Products,Services,Revenue,Operating income,imdb_id
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990.0,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,"[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...","Andrew ""Dice"" Clay",Oliver Wood,"[July 11, 1990, (, 1990-07-11, )]",102 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0098987
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990.0,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet","[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",NaN,Mark Plummer,"[May 17, 1990, (, 1990-05-17, ), (Cannes Film ...",114 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0098994
2,https://en.wikipedia.org/wiki/Air_America_(film),1990.0,https://www.imdb.com/title/tt0099005/,Air America,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",NaN,Roger Deakins,"[August 10, 1990, (, 1990-08-10, )]",113 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0099005
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990.0,https://www.imdb.com/title/tt0099012/,Alice,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",NaN,Carlo Di Palma,"[December 25, 1990, (, 1990-12-25, )]",106 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0099012
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990.0,https://www.imdb.com/title/tt0099018/,Almost an Angel,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",NaN,Russell Boyd,"December 19, 1990",95 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0099018


In [177]:
# Use a list comprehension to find out the number of values in each column that are null ----------------

# Return the column name, and sum of null values for each value (column) in the wiki_movies_df.columns iterable (columns of wiki_movies_df)
[[column,wiki_movies_df[column].isnull().sum()] for column in wiki_movies_df.columns]


[['url', 1],
 ['year', 1],
 ['imdb_link', 1],
 ['title', 6],
 ['Based on', 4872],
 ['Starring', 201],
 ['Narrated by', 6773],
 ['Cinematography', 711],
 ['Release date', 42],
 ['Running time', 150],
 ['Country', 245],
 ['Language', 259],
 ['Budget', 2318],
 ['Box office', 1572],
 ['Director', 20],
 ['Distributor', 378],
 ['Editor(s)', 567],
 ['Composer(s)', 537],
 ['Producer(s)', 218],
 ['Production company(s)', 1696],
 ['Writer(s)', 218],
 ['Genre', 6934],
 ['Original language(s)', 6892],
 ['Original network', 6925],
 ['Executive producer(s)', 6955],
 ['Production location(s)', 7006],
 ['Picture format', 6987],
 ['Audio format', 6992],
 ['Voices of', 7053],
 ['Followed by', 7045],
 ['Created by', 7042],
 ['Opening theme', 7054],
 ['No. of episodes', 7050],
 ['alt_titles', 7034],
 ['Preceded by', 7044],
 ['Author', 7052],
 ['Publisher', 7052],
 ['Publication date', 7052],
 ['Media type', 7053],
 ['Pages', 7054],
 ['ISBN', 7054],
 ['OCLC', 7054],
 ['LC Class', 7055],
 ['Cover artist', 7

In [178]:
# Several columns have over 6000 null values, making their inclusion largely pointless
# Using a list comprehension, we filter out columns which have MORE THAN 90% null values
# Alternatively stated, we INCLUDE ONLY COLUMNS WHICH HAVE LESS THAN 90% NULL VALUES
wiki_columns_to_keep = [column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9]
wiki_movies_df = wiki_movies_df[wiki_columns_to_keep]


In [179]:
list(wiki_movies_df.columns)

['url',
 'year',
 'imdb_link',
 'title',
 'Based on',
 'Starring',
 'Cinematography',
 'Release date',
 'Running time',
 'Country',
 'Language',
 'Budget',
 'Box office',
 'Director',
 'Distributor',
 'Editor(s)',
 'Composer(s)',
 'Producer(s)',
 'Production company(s)',
 'Writer(s)',
 'imdb_id']

In [180]:
# Regex Practice --------------------------------

# Regex only works on strings
# We can convert mismatched datatypes in columns using regular expressions

# Check which columns have mismatched datatypes

wiki_movies_df.dtypes


url                       object
year                     float64
imdb_link                 object
title                     object
Based on                  object
Starring                  object
Cinematography            object
Release date              object
Running time              object
Country                   object
Language                  object
Budget                    object
Box office                object
Director                  object
Distributor               object
Editor(s)                 object
Composer(s)               object
Producer(s)               object
Production company(s)     object
Writer(s)                 object
imdb_id                   object
dtype: object

Looking at the above output, we can note the following:
- year should be in datetime
- release date should be in datetime
- running time could be numeric
- budget should be an integer or float
- box office should be numeric

In [181]:
# Drop NaN values from the box office column and assign the series to a variable
box_office = wiki_movies_df['Box office'].dropna()


In [182]:
type(box_office[5341])

str

In [183]:
# Predefined function
def is_not_a_string(x):
    return type(x) != str

# Lambda equivalent
lambda x: type(x) != str

<function __main__.<lambda>(x)>

In [184]:
# Map the is_not_a_string function to the box_office series, to return the values that evaluate True
box_office[box_office.map(is_not_a_string)]

34                           [US$, 4,212,828]
56      [$6,698,361 (, United States, ), [2]]
77                    [$6,488,144, (US), [1]]
129                [US$1,531,489, (domestic)]
133                          [US$, 4,803,039]
                        ...                  
7212               [$99.6, million, [4], [5]]
7226                   [$365.6, million, [1]]
7227                         [$53.8, million]
7249                     [$435, million, [7]]
7283                   [$529.3, million, [4]]
Name: Box office, Length: 135, dtype: object

In [185]:
# Instead of using a predefined function and map, we can just pass in a lambda function into the map function 

box_office[box_office.map(lambda x: type(x) != str)]

34                           [US$, 4,212,828]
56      [$6,698,361 (, United States, ), [2]]
77                    [$6,488,144, (US), [1]]
129                [US$1,531,489, (domestic)]
133                          [US$, 4,803,039]
                        ...                  
7212               [$99.6, million, [4], [5]]
7226                   [$365.6, million, [1]]
7227                         [$53.8, million]
7249                     [$435, million, [7]]
7283                   [$529.3, million, [4]]
Name: Box office, Length: 135, dtype: object

In [186]:
# We can use the join() method to concatenate values in a list into a single value
# The join method has the following syntax: <'string to join items with'>.join(<iterable>). Iterable can be a list, tuple, or dict
# Docs: https://www.w3schools.com/python/ref_string_join.asp

# The following expression uses the apply() method: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.apply.html
# Applies the lambda function which joins values in each item in the series box_office, with a space character, if the item is a LIST
box_office = box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)
box_office[34]


'US$ 4,212,828'

In [187]:
# Create a regular expression to standardize currency values in our box_office column
form_one = r'\$\d+\.?\d*\s*[mb]illion'

# Check how many values match our first regular expression form
box_office.str.contains(form_one, flags=re.IGNORECASE).sum()


3895

In [188]:
# As you can see above, about 60% of records (3895/7000+) fit the above form
# Create a second regex to match another form of budget

form_two = "\$\d{1,3}(?:,\d{3})+"
box_office.str.contains(form_two, flags=re.IGNORECASE).sum()

1544

In [189]:
# Check which box office forms are matched by BOTH form one and form two

matches_form_one = box_office.str.contains(form_one, flags=re.IGNORECASE)
matches_form_two = box_office.str.contains(form_two, flags=re.IGNORECASE)

In [190]:
# this will throw an error!
box_office[(not matches_form_one) and (not matches_form_two)]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [35]:
# The above returns a ValueError because Pandas uses its own element-wise logical operators:
    # not = ~
    # and = &
    # or = |

# The following code is the above code using Pandas' logical operator nomenclature
# Check whether the values in the box_office series DO NOT MATCH EITHER FORM_ONE OR FORM_TWO
box_office[~matches_form_one & ~matches_form_two]


34                         US$ 4,212,828
82                              $335.000
113                   $4.35-4.37 million
133                        US$ 4,803,039
618                           $5000 (US)
756                         $ 11,146,270
993                             $ 50,004
1113                          35,254,617
1192    $ 407,618 (U.S.) (sub-total) [1]
1506                        $ 11,829,959
1541                          £3 million
1679                            $520.000
1952                        ¥1.1 billion
2132                                 N/A
2193                                $309
2234               US$ 171.8 million [9]
2365                   US$ 3,395,581 [1]
2371            $ 1,223,034 ( domestic )
2458                            $282.175
2757            $ 104,883 (US sub-total)
2784         926,423 admissions (France)
2817      $ 1.7 million (US) (sub-total)
2946                            $414.000
3050                            $621.000
3222           $

In [36]:
# Fix Pattern Matches -------------
# By looking at the Series above, we can see several patterns:
    # 1. Some values have a space between currency sign and number
    # 2. Some values use a period as a thousands separator, not comma
    # 3. Some values are a RANGE
    # 4. Million is sometimes spelled as "millon"

# if we go step-by-step, we can fix all of these patterns

# 1. Some values have a space between currency sign and number
    # Add \s* after dollar signs
    # signifies an unlimited amount of whitespace after the dollar sign (\s, *)
form_one = r'\$\s*\d+\.?\d*\s*[mb]illion'
form_two = r'\$\s*\d{1,3}(?:,\d{3})+'

# 2. Some values use a period as a thousands separator, not comma
    # Allow period OR comma as a thousands separator '[,\.]'
form_two = r'\$\s*\d{1,3}(?:[,\.]\d{3})+'

# Add a negative lookahead group to avoid matching values like 1.234 billion, which ARE valid
form_two = r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)'

# 3. Some values are a RANGE
    # Search for any string that starts with a dollar sign and ends with a hyphen and replace it with just a dollar sign
    # str.replace takes regex if we set the regex parameter to True

    # There are three different types of dashes in the square brackets below because of the different types of dashes (em dash, en dash, horizontal bar)
    # More info: https://en.wikipedia.org/wiki/Dash
box_office = box_office.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)

# 4. Million is sometimes spelled as "millon"
    # Make the second i in million optional using the ? 
form_one = r'\$\s*\d+\.?\d*\s*[mb]illi?on'


In [37]:
# Now that we have our types of forms, extract the data using the str.extract() method
# str.extract() can take a Regex and returns a DataFrame which matches the capture group(s)
# Pass in using f-string formatting both expressions as following:

box_office.str.extract(f'({form_one}|{form_two})')


,0
0,$21.4 million
1,$2.7 million
2,"$57,718,089"
3,"$7,331,647"
4,"$6,939,946"
...,...
7305,$19.4 million
7306,$41.9 million
7307,$76.1 million
7308,$38.4 million


In [ ]:
# Create a function to turn extracted values into a NUMERIC value:

def parse_dollars(s):
    # if s is not a string, return NaN

    # if input is of the form $###.# million

        # remove dollar sign and " million"

        # convert to float and multiply by a million

        # return value

    # if input is of the form $###.# billion

        # remove dollar sign and " billion"

        # convert to float and multiply by a billion

        # return value

    # if input is of the form $###,###,###

        # remove dollar sign and commas

        # convert to float

        # return value

    # otherwise, return NaN

In [38]:
def parse_dollars(s):
    # if s is not a string, return NaN
    if type(s) != str:
        return np.nan

    # if input is of the form $###.# million
    if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.IGNORECASE):

        # remove dollar sign and " million"
        s = re.sub('\$|\s|[a-zA-Z]','', s)

        # convert to float and multiply by a million
        value = float(s) * 10**6

        # return value
        return value

    # if input is of the form $###.# billion
    elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.IGNORECASE):

        # remove dollar sign and " billion"
        s = re.sub('\$|\s|[a-zA-Z]','', s)

        # convert to float and multiply by a billion
        value = float(s) * 10**9

        # return value
        return value

    # if input is of the form $###,###,###
    elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):

        # remove dollar sign and commas
        s = re.sub('\$|,','', s)

        # convert to float
        value = float(s)

        # return value
        return value

    # otherwise, return NaN
    else:
        return np.nan

In [39]:
# Extract the values from box_office using our two forms and the str.extract() method, then using the apply() method, apply our function parse_dollars
wiki_movies_df['box_office'] = box_office.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)


In [40]:
wiki_movies_df['box_office']

0       21400000.0
1        2700000.0
2       57718089.0
3        7331647.0
4        6939946.0
           ...    
7306    41900000.0
7307    76100000.0
7308    38400000.0
7309     5500000.0
7310           NaN
Name: box_office, Length: 7056, dtype: float64

In [41]:
# We don't need the box_office column anymore, so we can drop it
wiki_movies_df.drop('Box office', axis=1, inplace=True)


In [44]:
wiki_movies_df['box_office']

0       21400000.0
1        2700000.0
2       57718089.0
3        7331647.0
4        6939946.0
           ...    
7306    41900000.0
7307    76100000.0
7308    38400000.0
7309     5500000.0
7310           NaN
Name: box_office, Length: 7056, dtype: float64

In [85]:
# Parse budget data
# Assign budget column to a series and drop NaN values
budget = wiki_movies_df['Budget'].dropna()

# Concatenate items ONLY from a list into a string, separated by a space
budget = budget.map(lambda x: ' '.join(x) if type(x) == list else x)

# remove any values between a dollar sign and a hyphen (for budgets given in ranges) 
budget = budget.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)



In [86]:
# Using the form_one and form_two created above for the box_office data to parse the budget column and clean the values
matches_form_one = budget.str.contains(form_one, flags=re.IGNORECASE)
matches_form_two = budget.str.contains(form_two, flags=re.IGNORECASE)
budget[~matches_form_one & ~matches_form_two]

138                  USD$ 9 million
139                         Unknown
210     60 million Norwegian Kroner
262                        $ 80,000
362              US$ 65 million [1]
                   ...             
7047                  £12.9 million
7070                      3.5 crore
7124                        919,000
7133                    $8.6 millon
7305                   €4.3 million
Name: Budget, Length: 76, dtype: object

In [94]:
# Remove square bracket citations from our budget values

budget = budget.str.replace(r'\[\d+\]\s*', '')

# Check which values DO NOT match form_one and form_two (aka what remains)
budget[~matches_form_one & ~matches_form_two]

138                  USD$ 9 million
139                         Unknown
210     60 million Norwegian Kroner
262                        $ 80,000
362                 US$ 65 million 
                   ...             
7047                  £12.9 million
7070                      3.5 crore
7124                        919,000
7133                    $8.6 millon
7305                   €4.3 million
Name: Budget, Length: 70, dtype: object

['USD$ 9 million',
 'Unknown',
 '60 million Norwegian Kroner',
 '$ 80,000',
 'US$ 65 million ',
 'US$ 5 million',
 'US$ 24 million',
 'Unknown',
 '$ 30 million',
 'BEF 1 million ( USD$ 33,000)',
 '$ 70 million ',
 '$ 3,000',
 'Unknown',
 'HBO',
 '£6,000,000',
 '13 million',
 '$ 24,000,000',
 '£2.8 million',
 'CAD2,000,000',
 '$ 15,000,000 (estimated)',
 'PHP 85 million (estimated)',
 '102,888,900',
 '3,500,000 DM',
 '$ 10,000,000 (approximately)',
 '₤2,300,874',
 'US$ 1 million',
 '$14 milion',
 'USD$ 10,000,000 (est.)',
 '$ 38 million ',
 'CAD$ 31 million ( USD$ 15 million )',
 'US$ 2,600,000',
 '₤6,350,000',
 '$ 25 million ',
 '$ 3 million (est.)',
 '$ 115 million ',
 '€ 40 million',
 'US$ 30 million',
 'US$ 855,000 ',
 '$ 400,000',
 '$ 500,000',
 '$218.32',
 '£4.2 million',
 'CN¥156 million ( US$ 20 million)',
 'N/A',
 '$ 2,800,000 (Estimate)',
 '760,000 USD',
 'US$ 3,000,000',
 '€ 1 million (approx. USD$ 1.3 million)',
 '$ 105,000',
 'US$ 10 million',
 '19 crore',
 '£17 million',
 

In [129]:
# Reuse our code from the box_office column for extracting budget data, applying the parse_dollars to convert data into float
wiki_movies_df['budget'] = budget.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)

# Drop the original 'Budget' column
wiki_movies_df.drop('Budget', axis=1, inplace=True)

In [130]:
# Parse release date
# Drop null values and re-assign all values to string datatype

release_date = wiki_movies_df['Release date'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)


In [138]:
# Full month name, one- to two-digit day, four-digit year (i.e., January 1, 2000)
# Four-digit year, two-digit month, two-digit day, with any separator (i.e., 2000-01-01)
# Full month name, four-digit year (i.e., January 2000)
# Four-digit year

date_form_one = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s[123]\d,\s\d{4}'
date_form_two = r'\d{4}.[01]\d.[123]\d'
date_form_three = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}'
date_form_four = r'\d{4}'


In [ ]:
release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})', flags=re.IGNORECASE)


In [139]:
# Use pandas datetime module to convert values to datetime, using the infer_datetime_format=True argument to recognize any of the four types we have identified
wiki_movies_df['release_date'] = pd.to_datetime(release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})')[0], infer_datetime_format=True)


In [213]:
# Parse running time ---------------------------------
running_time = wiki_movies_df['Running time'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)

In [214]:
# Count the number of values in the running_time Series that fit the format of "x minutes"
running_time.str.contains(r'^\d*\s*minutes$', flags=re.IGNORECASE).sum()


6537

In [204]:
# Inspect the remaining data that does not match the above filter 
running_time[running_time.str.contains(r'^\d*\s*minutes$', flags=re.IGNORECASE) != True]


9                                                 102 min
26                                                 93 min
28                                                32 min.
34                                                101 min
35                                                 97 min
                              ...                        
6719       114 minutes [1] 120 minutes (extended edition)
6866                                             104 mins
6934    90 minutes (theatrical) [1] 91 minutes (unrate...
7292    108 minutes (Original cut) 98 minutes (UK cut)...
7310                Variable; 90 minutes for default path
Name: Running time, Length: 369, dtype: object

In [192]:
# 369 values do not match the 'x minutes' format
# Adjust the RegEx to factor in ABBREVIATIONS of minutes, such as 'min.'
running_time.str.contains(r'^\d*\s*m', flags=re.IGNORECASE).sum()


6888

In [205]:
running_time[running_time.str.contains(r'^\d*\s*m', flags=re.IGNORECASE) != True]


651                                                  3:52
689                     UK:84 min (DVD version) US:86 min
751                         78-102 min (depending on cut)
869                       Varies (79 [3] –84 [1] minutes)
1400                                              25 : 03
1503    United States: 77 minutes Argentina: 94 minute...
1560                                            1hr 35min
1615                                               varies
1859                    Netherlands:96 min, Canada:95 min
1862                                       approx. 14 min
2382                                           1 h 43 min
3123                                               1h 48m
4074                                              4 hours
4599    US domestic version: 86 minutes Original versi...
5156    Theatrical cut: 97 minutes Unrated cut: 107 mi...
5621                    115 [1] /123 [2] /128 [3] minutes
5645                                    1 hour 32 minutes
7310          

In [220]:
# Remove the caret in front of \d which previously indicated that the digits had to be at the start of the string
# Count the number of values that DO NOT MATCH the following RegEx
running_time[running_time.str.contains(r'\d*\s*m', flags=re.IGNORECASE) != True]

pandas.core.series.Series

In [215]:
# Expand regex to match multiple patterns - str.extract() -------------------
# str.extract() function is called on a Series and takes a regex and extracts the capturing groups that match ('') into columns of a Dataframe

running_time_extract = running_time.str.extract(r'(\d+)\s*ho?u?r?s?\s*(\d*)|(\d+)\s*m')
running_time_extract


,0,1,2
0,NaN,NaN,102
1,NaN,NaN,114
2,NaN,NaN,113
3,NaN,NaN,106
4,NaN,NaN,95
...,...,...,...
7306,NaN,NaN,90
7307,NaN,NaN,132
7308,NaN,NaN,120
7309,NaN,NaN,123


In [216]:
# To each column in our extracted dataframe, convert the values into numeric values using the to_numeric() method
# errors='coerce' converts errors into NaN, which we can then chain the fillna() method to replace with '0'
running_time_extract = running_time_extract.apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)


In [221]:
# Convert the hour and minute capture groups to minutes if the pure minutes capture group is 0
# Axis=1 sets the indices to the dataframe's columns, instead of the dataframe's indices (ie column 1 is 0, instead of a separate index column)
wiki_movies_df['running_time'] = running_time_extract.apply(lambda row: row[0]*60 + row[1] if row[2] == 0 else row[2], axis=1)


pandas.core.frame.DataFrame